### This notebook shows examples of how to use the cyclops.query API on MIMIC-IV.

## Example 1. Get all patient encounters from 2015.

In [1]:
from sqlalchemy import select

from cyclops.query import mimic
from cyclops.query import utils as query_utils
from cyclops.query.interface import QueryInterface

patients = mimic.patients(years=[2015])
patients.run()
print(f"{len(patients.data)} rows extracted!")

2022-04-04 15:03:18,205 INFO cyclops.config  - Log file is /home/amritk/projects/cyclops/log.log
2022-04-04 15:03:18,481 INFO cyclops.utils.profile - Log file is /home/amritk/projects/cyclops/log.log
2022-04-04 15:03:18,483 INFO cyclops.query.utils - Log file is /home/amritk/projects/cyclops/log.log
2022-04-04 15:03:18,484 INFO cyclops.orm     - Log file is /home/amritk/projects/cyclops/log.log
2022-04-04 15:03:19,271 INFO cyclops.orm     - Database setup, ready to run queries!
2022-04-04 15:03:19,853 INFO cyclops.orm     - Query returned successfully!
2022-04-04 15:03:19,854 INFO cyclops.utils.profile - Finished executing function run_query in 0.572805 s


88488 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in ICD long title), in the year 2015.

In [2]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses(substring="schizophrenia")
patients_schizophrenia = mimic.join_with_patients(patients.query, diagnoses.query)
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")
patients_schizophrenia.data["icd_title"].value_counts()

2022-04-04 15:03:20,482 INFO cyclops.orm     - Query returned successfully!
2022-04-04 15:03:20,483 INFO cyclops.utils.profile - Finished executing function run_query in 0.431260 s


802 rows extracted!


Schizophrenia, unspecified                                                 380
Unspecified schizophrenia, unspecified                                     179
Paranoid schizophrenia                                                      77
Paranoid type schizophrenia, unspecified                                    51
Catatonic schizophrenia                                                     21
Unspecified schizophrenia, chronic                                          17
Paranoid type schizophrenia, chronic                                        12
Paranoid type schizophrenia, chronic with acute exacerbation                12
Other schizophrenia                                                         10
Other specified types of schizophrenia, unspecified                          8
Catatonic type schizophrenia, unspecified                                    7
Unspecified schizophrenia, chronic with acute exacerbation                   4
Catatonic type schizophrenia, chronic with acute exa

## Example 3. Advanced - uses regexp_match from sqlalchemy. Get all patient encounters with diagnoses (ICD long title begins with `schizophrenia` ), in the year 2015.

In [3]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses()
subquery = (
    select(diagnoses.query)
    .where(diagnoses.query.c.icd_title.regexp_match("(?=.*schizophrenia)(?=.*chronic)"))
    .subquery()
)
patients_schizophrenia = mimic.join_with_patients(patients.query, subquery)
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")

2022-04-04 15:03:23,436 INFO cyclops.orm     - Query returned successfully!
2022-04-04 15:03:23,437 INFO cyclops.utils.profile - Finished executing function run_query in 1.041602 s


57 rows extracted!


## Example 4. Advanced - uses cyclops.query.utils.starts_with helper function. Get all patient encounters with diagnoses (starts with `schizophrenia` in ICD long title), in the year 2015.

In [4]:
patients = mimic.patients(years=[2015])
diagnoses = mimic.diagnoses()
subquery = (
    select(diagnoses.query)
    .where(query_utils.starts_with(diagnoses.query.c.icd_title, "schizophrenia"))
    .subquery()
)
patients_schizophrenia = mimic.join_with_patients(patients.query, subquery)
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")

2022-04-04 15:03:25,831 INFO cyclops.orm     - Query returned successfully!
2022-04-04 15:03:25,831 INFO cyclops.utils.profile - Finished executing function run_query in 0.422913 s


380 rows extracted!


## Example 5. Get all patient encounters with diagnoses (ICD code is F209).

In [7]:
patients = mimic.patients()
diagnoses = mimic.diagnoses(diagnosis_codes=["F209"])
patients_schizophrenia = mimic.join_with_patients(patients.query, diagnoses.query)
patients_schizophrenia.run()
print(f"{len(patients_schizophrenia.data)} rows extracted!")

2022-04-04 15:03:53,428 INFO cyclops.orm     - Query returned successfully!
2022-04-04 15:03:53,429 INFO cyclops.utils.profile - Finished executing function run_query in 1.348303 s


1442 rows extracted!


## Example 6. Get all patient encounters with diagnoses (`delirium` in ICD long title).

In [8]:
patients = mimic.patients()
diagnoses = mimic.diagnoses(substring="delirium")
patients_delirium = mimic.join_with_patients(patients.query, diagnoses.query)
patients_delirium.run()
print(f"{len(patients_delirium.data)} rows extracted!")

2022-04-04 15:06:56,415 INFO cyclops.orm     - Query returned successfully!
2022-04-04 15:06:56,415 INFO cyclops.utils.profile - Finished executing function run_query in 0.699475 s


9087 rows extracted!


## Labs

In [ ]:
query = select(db.mimic_icu.d_items.data)
db.run_query(query, limit=5)

In [ ]:
query = qm.event_labels(db, category="Routine Vital Signs")
db.run_query(query, limit=5)

In [ ]:
query = qm.event_labels_by_substring(db, "sodium")
db.run_query(query, limit=5)

In [ ]:
query = qm.events(db)
db.run_query(query, limit=3)

In [ ]:
query = qm.events_by_itemids(db, 223876)
db.run_query(query, limit=5)

In [ ]:
query = qm.events_by_itemids(db, [223873, 223875])
db.run_query(query, limit=5)

In [ ]:
query = qm.events_by_labels(db, ["Paw High"])
db.run_query(query, limit=5)

In [ ]:
query = qm.events_by_label_substring(db, "heart")
db.run_query(query, limit=5)

In [ ]:
db.run_query(db.mimic_icu.chartevents, limit=5)

In [ ]:
db.run_query(db.mimic_icu.d_items.data, limit=5)

In [ ]:
query = qm.events(db, category="Routine Vital Signs")
db.run_query(query, limit=3)

In [ ]:
query = qm.events(db, category="Respiratory")
db.run_query(query, limit=3)

In [ ]:
from datetime import datetime

# Get heart rate
query = qm.events_by_labels(db, "Heart Rate")
df = db.run_query(query, limit=1000)

# Get only the labs with numeric value - This is just for now
# realistically we can find ways to utilize the other data!
df = df[df["valuenum"].notna()]

# Convert to datetime
df["charttime"] = pd.to_datetime(df["charttime"])

# Group by patient (subject_id), admission (hadm_id), ICU stay (stay_id), and event (itemid)
# NOTE: We may or may not want to group by hadm_id and/or stay_id
groupby = df.groupby(["subject_id", "hadm_id", "stay_id", "itemid"])

c = 0
for name, group in groupby:
    group.sort_values(by="charttime", inplace=True)

    label = group["label"].iloc[0]

    fig, ax = plt.subplots(1, 2, figsize=(15, 5))

    ax[0].set_title(label)
    ax[0].plot(group["charttime"], group["valuenum"])
    ax[0].set_ylabel(label)
    ax[0].set_xlabel("Datetime")

    # Convert dates to seconds elapsed since first measurement
    group["charttime"] = (group["charttime"] - datetime.now()).dt.total_seconds()
    group["charttime"] -= group["charttime"].min()

    ax[1].set_title(label)
    ax[1].plot(group["charttime"], group["valuenum"])
    ax[1].scatter(group["charttime"], group["valuenum"])
    ax[1].set_ylabel(label)
    ax[1].set_xlabel("Seconds elapsed")
    c += 1
    if c > 10:
        break